# Install and load packages

In [ ]:
!pip install factor_analyzer

In [ ]:
!pip install scikit-learn==0.24.2

In [ ]:
!pip install nolds

In [ ]:
!git clone https://github.com/josemiotto/pylevy

# navigate to atalaia directory
%cd pylevy

# get modifications made on the repo
!git pull origin master

# install packages requirements
#!pip install -r requirements.txt

# install package
!python setup.py install

In [ ]:
%cd /content

In [ ]:
# Import packages
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import norm, gumbel_l
import sklearn as skl
from sklearn.linear_model import LinearRegression 
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.decomposition import PCA, FactorAnalysis
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
import seaborn as sb
import factor_analyzer as fa
from nolds import hurst_rs
import levy

# Preparation

In [ ]:
NumSim = 200 #Number of Simulations
t = 250 #time, i.e. number of observations
n = 600 #number of time series

features = ['std', 'skew', 'kurt', 'VaR0.05', 'ES', 'beta', 'VaR0.95', 'EU', 'autocor', 'Hurst', 'stab_a', 'stab_g']
num_feat = len(features) #number of features 
num_clust = 3 #number of clusters: in simulation we know the true value. With real world data we have to try different ones.


name_models = ['pca2_km', 'pca3_km', 'pca2_ac', 'pca3_ac',
               'fa2_km', 'fa3_km', 'fa2v_km', 'fa3v_km',
               'fa2_ac', 'fa3_ac', 'fa2v_ac', 'fa3v_ac']
NumModels = len(name_models) #number of models

In [ ]:
y_true = np.zeros(n, dtype=int)
y_true[int(n/3):int(2*n/3)]= 1
y_true[int(2*n/3):] = 2

#To store the features
Data_save = np.zeros((NumSim, n+1, num_feat))

#Adj_Rand: to save the Adj_Rand_Scores
Adj_Rand = pd.DataFrame(np.zeros((NumSim, NumModels)), columns = name_models)

In [ ]:
def km(model, X, y):
  kmeans = model.fit(X)
  yhat = kmeans.predict(X)
  a = adjusted_rand_score(y, yhat)
  return a

def agc(X, y):
  yhat = AgglomerativeClustering(n_clusters=num_clust).fit_predict(X)
  a = adjusted_rand_score(y, yhat)
  return a

# Simulate data and calculate features

In [ ]:
for i in range(NumSim):
  TSData = np.zeros((t, n+1)) #n+1 because we need market proxy TS
  for j in range(int(n/3)):
    TSData[:,j] = norm.rvs(loc=0, scale=0.7, size=t) #loc=mean, scale=standard deviation
    TSData[:,j+int(n/3)] = norm.rvs(loc=0, scale=0.2, size=t)
    TSData[:,j+2*int(n/3)] = gumbel_l.rvs(size=t)
    TSData[:, n] = np.mean(TSData[:, :n], axis=1) #market proxy TS: average over all

  TSDatanp = TSData
  TSData = pd.DataFrame(TSData)
  #Calculate features
  Data = pd.DataFrame(np.zeros((n+1, num_feat)), columns = features)
  
  Data['std'] = TSData.std(axis=0)
  Data['skew'] = TSData.skew(axis=0)
  Data['kurt'] = TSData.kurtosis(axis=0)
  Data['VaR0.05'] = TSData.quantile(q=0.05, axis=0)
  Data['VaR0.95'] = TSData.quantile(q=0.95, axis=0)
  Data['ES'] = TSData[TSData  < Data['VaR0.05']].mean(axis=0)
  Data['EU'] = TSData[TSData  > Data['VaR0.95']].mean(axis=0)
  
  #CAPM beta
  r_market = TSDatanp[:,n].reshape(t,1)
  for j in range(n+1):
    X = TSDatanp[:,j].reshape(t,1)
    reg = LinearRegression().fit(X, r_market)
    Data.loc[j,'beta']=reg.coef_
    #Data.loc[j,'alpha']=reg.intercept_ #alpha only if necessary

  # 'autocor', 'Hurst', 'stab_a', 'stab_g'
  for j in range(n+1):
    series = TSData.iloc[:,j]
    #autocorrelation coefficient
    Data.loc[j,'autocor'] = series.autocorr()
    #Hurst
    Data.loc[j, 'Hurst'] = hurst_rs(series) #using nolds package
    #fit levy stable distribution
    levystab = levy.fit_levy(series)
    #alpha stable
    Data.loc[j, 'stab_a'] = levystab[0].get()[0]
    #gamma stable
    Data.loc[j, 'stab_g'] = levystab[0].get()[3]

  Data_save[i,:,:] = Data

# Compute PCA/Factor Analysis and calculate Adjusted Rand Index

In [ ]:
for i in range(NumSim):
  Data = pd.DataFrame(Data_save_new[i,:,:], columns=features)
  #Scale the data
  X = StandardScaler().fit_transform(Data.loc[:n-1, :])

  #Clustering
  km_model = KMeans(n_clusters=3)

  #PCA (in this case necessary to use scaled data), then Clustering
  pca2 = PCA(2)
  pca3 = PCA(3)
  Y_2 = pca2.fit_transform(X)
  Y_3 = pca3.fit_transform(X)

  Adj_Rand.loc[i, 'pca2_km'] = km(km_model, Y_2, y_true)
  Adj_Rand.loc[i, 'pca3_km'] = km(km_model, Y_3, y_true)
  Adj_Rand.loc[i, 'pca2_ac'] = agc(Y_2, y_true)
  Adj_Rand.loc[i, 'pca3_ac'] = agc(Y_3, y_true)

  #Factor Analysis, then clustering
  fac2 = FactorAnalysis(n_components=2)#Factor Analysis with 2 factors
  fac3 = FactorAnalysis(n_components=3)#Factor Analysis with 3 factors
  fac2_vari = FactorAnalysis(n_components=2, rotation = 'varimax')#FA with 2 factors and Varimax rotation
  fac3_vari = FactorAnalysis(n_components=3, rotation = 'varimax')#FA with 3 factors and Varimax rotation

  F_2 = fac2.fit_transform(X)
  F_3 = fac3.fit_transform(X)
  F_2V = fac2_vari.fit_transform(X)
  F_3V = fac3_vari.fit_transform(X)
  
  Adj_Rand.loc[i, 'fa2_km'] = km(km_model, F_2, y_true)
  Adj_Rand.loc[i, 'fa3_km'] = km(km_model, F_3, y_true)
  Adj_Rand.loc[i, 'fa2v_km'] = km(km_model, F_2V, y_true)
  Adj_Rand.loc[i, 'fa3v_km'] = km(km_model, F_3V, y_true)
  Adj_Rand.loc[i, 'fa2_ac'] = agc(F_2, y_true)
  Adj_Rand.loc[i, 'fa3_ac'] = agc(F_3, y_true)
  Adj_Rand.loc[i, 'fa2v_ac'] = agc(F_2V, y_true)
  Adj_Rand.loc[i, 'fa3v_ac'] = agc(F_3V, y_true)